In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()

import sys
import os

COCO_DATA = '../data/coco'
MASK_RCNN_MODEL_PATH = '../lib/Mask_RCNN/'
SIAMESE_MASK_RCNN_PATH = '../lib/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
from samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
import utils as siamese_utils
import model as siamese_model
import config as siamese_config
   
import time
import datetime
import random
import numpy as np
import skimage.io
import imgaug
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## Dataset

In [ ]:
# Load COCO/val dataset
test_dataset = siamese_utils.IndexedCocoDataset()
# coco_val.set_active_classes(train_classes)
coco_test_object = test_dataset.load_coco(COCO_DATA, subset="val", year="2017", return_coco=True)
test_dataset.prepare()
test_dataset.build_indices()

## Model

In [ ]:
class ParallelTrainConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1
    NUM_TARGETS = 5
    NAME = 'coco'
    EXPERIMENT = 'evaluation'
    
    # Large image sizes
    TARGET_MAX_DIM = 192
    TARGET_MIN_DIM = 150
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024
    # Large model size
    FPN_CLASSIF_FC_LAYERS_SIZE = 1024
    FPN_FEATUREMAPS = 256
    # Large number of rois at all stages
    RPN_ANCHOR_STRIDE = 1
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    POST_NMS_ROIS_TRAINING = 2000
    POST_NMS_ROIS_INFERENCE = 1000
    TRAIN_ROIS_PER_IMAGE = 200
    DETECTION_MAX_INSTANCES = 100
    MAX_GT_INSTANCES = 100
    
class SmallTrainConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 1
    NAME = 'coco'
    EXPERIMENT = 'evaluation'

In [ ]:
config = ParallelTrainConfig()

In [ ]:
# Provide training schedule of the model
# When evaluationg intermediate steps the tranining schedule must be provided
short_train_schedule = OrderedDict()
short_train_schedule[1] = {"learning_rate": config.LEARNING_RATE, "layers": "heads"}
short_train_schedule[120] = {"learning_rate": config.LEARNING_RATE, "layers": "all"}
short_train_schedule[160] = {"learning_rate": config.LEARNING_RATE/10, "layers": "all"}

# Provide training schedule of the model
# When evaluationg intermediate steps the tranining schedule must be provided
long_train_schedule = OrderedDict()
long_train_schedule[1] = {"learning_rate": config.LEARNING_RATE, "layers": "heads"}
long_train_schedule[240] = {"learning_rate": config.LEARNING_RATE, "layers": "all"}
long_train_schedule[320] = {"learning_rate": config.LEARNING_RATE/10, "layers": "all"}

## Siamese Mask R-CNN

### All 80 classes

In [ ]:
# Load pretrained checkpoint
checkpoint = '../checkpoints/large_siamese_mrcnn_coco_full_0320.h5'

In [ ]:
ckpt_dir = os.path.join(ROOT_DIR, 'logs/parallel_coco_full')
config = ParallelTrainConfig()

# Select checkpoint
step = 320 # usually the last ckpt is the best
checkpoint = os.path.join(ckpt_dir, 'siamese_mrcnn_0{:0>3}.h5'.format(step))

In [ ]:
# Evaluate on all classes
test_dataset.ACTIVE_CLASSES = np.array(range(1,81))

# Load and evaluate models

# Create model object in inference mode.
model = siamese_model.SiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_checkpoint(checkpoint, training_schedule=long_train_schedule)
# Evaluate only active classes
active_class_idx = np.array(test_dataset.ACTIVE_CLASSES) - 1

# Evaluate on the validation set
print('evaluating five times')

for run in range(5):
    print('\t*** Evaluation run {} ***'.format(run + 1))
    
    siamese_utils.evaluate_dataset(model, test_dataset, coco_test_object, eval_type=["bbox", "segm"], 
                     dataset_type='coco', limit=0, image_ids=None, # limit=0 -> entire data set
                     class_index=active_class_idx, verbose=1)
    
    print('\n' * 5, end='')

### i-split

In [ ]:
index = 1 # between 1 and 4

In [ ]:
one_shot_classes = np.array([4*i + index for i in range(20)])
train_classes = np.array(range(1,81))[np.array([i not in one_shot_classes for i in range(1,81)])]

In [ ]:
# Load pretrained checkpoint
config = ParallelTrainConfig()
checkpoint = '../checkpoints/large_siamese_mrcnn_coco_i{}_0160.h5'.format(index)

In [ ]:
# Load self-trained checkpoint
ckpt_dir = os.path.join(ROOT_DIR, 'logs/parallel_coco_i{}'.format(index))
config = ParallelTrainConfig()

# Select checkpoint
step = 160 # usually the last ckpt is the best
checkpoint = os.path.join(ckpt_dir, 'siamese_mrcnn_0{:0>3}.h5'.format(step))

#### train classes

In [ ]:
# Evaluate on training classes
test_dataset.ACTIVE_CLASSES = train_classes

# Load and evaluate models

# Create model object in inference mode.
model = siamese_model.SiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_checkpoint(checkpoint, training_schedule=short_train_schedule)
# Evaluate only active classes
active_class_idx = np.array(test_dataset.ACTIVE_CLASSES) - 1

# Evaluate on the validation set
print('evaluating five times')

for run in range(5):
    print('\t*** Evaluation run {} ***'.format(run + 1))
    
    siamese_utils.evaluate_dataset(model, test_dataset, coco_test_object, eval_type=["bbox", "segm"], 
                     dataset_type='coco', limit=0, image_ids=None, 
                     class_index=active_class_idx, verbose=1)
    
    print('\n' * 5, end='')

#### one-shot classes

In [ ]:
# Evaluate on one-shot classes
test_dataset.ACTIVE_CLASSES = one_shot_classes

# Load and evaluate models

# Create model object in inference mode.
model = siamese_model.SiameseMaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_checkpoint(checkpoint, training_schedule=short_train_schedule)
# Evaluate only active classes
active_class_idx = np.array(test_dataset.ACTIVE_CLASSES) - 1

# Evaluate on the validation set
print('evaluating five times')

for run in range(5):
    print('\t*** Evaluation run {} ***'.format(run + 1))
    
    siamese_utils.evaluate_dataset(model, test_dataset, coco_test_object, eval_type=["bbox", "segm"], 
                     dataset_type='coco', limit=0, image_ids=None, 
                     class_index=active_class_idx, verbose=1)
    
    print('\n' * 5, end='')